In [0]:
%python
%pip install py7zr

In [0]:
import py7zr

full_path_7z = "/Volumes/demos_standard/public_data/public_raw/airport-codes.7z"
extract_dir = "/Volumes/demos_standard/public_data/public_raw/tmp/airport_codes_extract"

with py7zr.SevenZipFile(full_path_7z, mode='r') as archive:
    archive.extractall(path=extract_dir)

import os
csv_file = [f for f in os.listdir(extract_dir) if f.endswith('.csv')][0]
csv_path = os.path.join(extract_dir, csv_file)

df_all = spark.read.option("header", "true").option("sep", ",").csv(csv_path)
print(df_all.count())

In [0]:
print(f"Total records: {df_all.count()}")
print(f"Distinct ident count: {df_all.select('ident').distinct().count()}")

In [0]:
display(df_all.limit(12))

In [0]:
from pyspark.sql.functions import lower, when, col

df_t_count = (
    df_all.groupBy("type")
    .count()
    .withColumn(
        "AirportSize",
        when(lower(col("type")).contains("closed"), "Closed")
        .when(lower(col("type")).contains("large"), "Large")
        .when(lower(col("type")).contains("medium"), "Medium")
        .otherwise("Small")
    )
)
display(df_t_count)

In [0]:
df_c_count = df_all.groupBy("continent","iso_country").count()
display(df_c_count)

In [0]:
df_r_count = df_all.groupBy("iso_region","continent").count().orderBy("continent","iso_region")
display(df_r_count)